In [1]:
from pyspark.sql.functions import col,monotonically_increasing_id,when,lag
sc.install_pypi_package("pandas==0.25.1") 
sc.install_pypi_package("s3fs==0.4.0") 


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1575218512374_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


  Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1

In [2]:
# File location and type
file_location = "s3://gps-yanzhaoliang/all.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.format(file_type).option("inferSchema", infer_schema).option("header", first_row_is_header).option("sep", delimiter).load(file_location)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
file_location = "s3://gps-yanzhaoliang/all1.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df2 = spark.read.format(file_type).option("inferSchema", infer_schema).option("header", first_row_is_header).option("sep", delimiter).load(file_location)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
file_location = "s3://gps-yanzhaoliang/all2.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df3 = spark.read.format(file_type).option("inferSchema", infer_schema).option("header", first_row_is_header).option("sep", delimiter).load(file_location)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df0 = df1.union(df2)
df=df0.union(df3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df =df.withColumn("idx", monotonically_increasing_id())
# because all the client names have been hashed, so, i need to some conversion to make the the client name more clearly.


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="client", outputCol="client_with_order")
indexed = indexer.fit(df).transform(df)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_1 = indexed
#from pyspark.sql.types import StringType
df_2 = df_1.withColumn("client_order",df_1["client_with_order"].cast("int"))
df_2 = df_2.withColumn("client_order",df_2["client_order"].cast("string"))
from pyspark.sql.functions import lit, concat,col
#df_2 = df_2.withColumn("client_1",lit("client"))
df_2 = df_2.withColumn('client1', concat( lit('client'), col('client_order')))
df_2=df_2.withColumnRenamed('client', 'client_df_2')
df_3=df_2.drop('client_with_order','client_order','client_1')
df_3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+----------+---+-----------+
|          timestamp|         client_df_2|        AP|idx|    client1|
+-------------------+--------------------+----------+---+-----------+
|2014-01-01 00:00:24|b7f22f3abd9f367af...|Bldg11AP21|  0| client2279|
|2014-01-01 00:00:30|336501aeaedff0462...| Bldg44AP3|  1|   client59|
|2014-01-01 00:00:35|4b912f490ddc1f289...|Bldg48AP65|  2| client4546|
|2014-01-01 00:00:40|8e10f9d5aa9c229ba...|Bldg25AP19|  3|client11368|
|2014-01-01 00:00:51|680fc4c2bd6b252fd...| Bldg25AP5|  4|   client38|
|2014-01-01 00:00:59|d87196dde7bad5815...|Bldg14AP12|  5|client13610|
|2014-01-01 00:01:05|980b004cef94929b5...|Bldg16AP22|  6|  client726|
|2014-01-01 00:01:15|e7e4501b7195d38ec...|Bldg3AP134|  7|client87752|
|2014-01-01 00:01:20|d8891a295b4343f5a...| Bldg44AP2|  8|   client84|
|2014-01-01 00:01:20|e7e4501b7195d38ec...|Bldg3AP134|  9|client87752|
|2014-01-01 00:01:47|dec63a259f5158845...|Bldg48AP29| 10| client3911|
|2014-01-01 00:01:53

In [9]:
from pyspark.sql.functions import split
split_col = split(df_3['AP'], 'AP')
df_3 = df_3.withColumn('building', split_col.getItem(0))
df_3.show()
#df.filter(df.contains.like('%google.com%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+----------+---+-----------+--------+
|          timestamp|         client_df_2|        AP|idx|    client1|building|
+-------------------+--------------------+----------+---+-----------+--------+
|2014-01-01 00:00:24|b7f22f3abd9f367af...|Bldg11AP21|  0| client2279|  Bldg11|
|2014-01-01 00:00:30|336501aeaedff0462...| Bldg44AP3|  1|   client59|  Bldg44|
|2014-01-01 00:00:35|4b912f490ddc1f289...|Bldg48AP65|  2| client4546|  Bldg48|
|2014-01-01 00:00:40|8e10f9d5aa9c229ba...|Bldg25AP19|  3|client11368|  Bldg25|
|2014-01-01 00:00:51|680fc4c2bd6b252fd...| Bldg25AP5|  4|   client38|  Bldg25|
|2014-01-01 00:00:59|d87196dde7bad5815...|Bldg14AP12|  5|client13610|  Bldg14|
|2014-01-01 00:01:05|980b004cef94929b5...|Bldg16AP22|  6|  client726|  Bldg16|
|2014-01-01 00:01:15|e7e4501b7195d38ec...|Bldg3AP134|  7|client87752|   Bldg3|
|2014-01-01 00:01:20|d8891a295b4343f5a...| Bldg44AP2|  8|   client84|  Bldg44|
|2014-01-01 00:01:20|e7e4501b7195d38ec...|Bldg3AP134

In [10]:
check_in_times=df_3.groupby("client1")
from pyspark.sql.functions import desc

# check_in_times=check_in_times.count().filter("`count` >= 1").sort(desc("count"))
# #filter("`count` >= 10").
# check_in_times.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# check_in_times=check_in_times.toPandas()
from pyspark.sql.functions import desc
check_in_times=check_in_times.count().filter("`count` >= 10000").filter("`count` <= 20000").sort(desc("count"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
list_checkin=check_in_times.select('client1').distinct().collect()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:

list_checkinids=[list(i.asDict().values())[0] for i in list_checkin]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
dfArray = [df_3.where(df_3.client1 == x) for x in list_checkinids]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
for i in range(318,1400):
    df_client=dfArray[i]
    df_client.toPandas().to_csv("s3://gps-yanzhaoliang/markov/client"+str(i)+".csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from https://172.31.89.105:18888/sessions/0/statements/15 with error payload: {"msg":"requirement failed: Session isn't active."}
